In [1]:
using Revise
#using ProgressBars
using Random
using ITensors
#using Pkg;Pkg.activate("../.")
using RandomMeas
using StatsBase
using TimerOutputs
using ProgressMeter

Precompiling RandomMeas
  ✓ RandomMeas
  1 dependency successfully precompiled in 5 seconds. 275 already precompiled.


## Shallow Shadows 
Based on https://quantum-journal.org/papers/q-2023-06-01-1026 and https://doi.org/10.1103/PhysRevResearch.5.023027 and arXiv:2402.17911

In [160]:
#Step 1: Calibration Data acquisition
N = 4
ξ = siteinds("Qubit", N;addtags="output")
χ = 3
χchannel = 16
Nu = 400
states = ["Dn" for n in 1:N]
ψ0 = MPS(ξ,states);

### Introduction
For a given set of random unitaries, the measurement channel is defined as $\mathcal{M}(\rho)=\sum_s \bra{s}u\rho u^\dagger \ket{s}E[u^\dagger \ket{s}\bra{s}u]$. To design a randomized measurement protocol, we need to learn this channel, then invert it. Once we know the inverse channel, we will be able to form shadows on arbitrary states $\rho$, as $\hat{\rho}=\mathcal{M}^{-1}(u^\dagger \ket{s}\bra{s}u)$

We begin by picking a set of $N_u$ unitaries and calculate numerically the measurement channel $\mathcal{M}(\rho)$ from MPS simulations. Note that alternatively, one could consider measuring an approximate representation of the channel experimentally (arXiv:2402.17911)

In [161]:
u = Vector{Vector{ITensor}}()
for r in 1:Nu
    u1 = [op("RandomUnitary",ξ[i]) for i in 1:N]
    #push!(u,u1)
    u2 = [op("RandomUnitary",ξ[i],ξ[i+1]) for i in 1:N-1]
    u3 = [op("RandomUnitary",ξ[i],ξ[i+1]) for i in 2:N-2]
    push!(u,[u1;u2;u3])
end
M = EvaluateMeasurementChannel(ψ0,u,χchannel);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:04


In [162]:
nsweeps = 6
σ = FitChannelMPO(M,χ,nsweeps);

env done
Lower bound to distance 0.040424830916116226
Lower bound to distance -0.06907018232615081
Lower bound to distance -0.06907216766374834
Lower bound to distance -0.06907326884906059
Lower bound to distance -0.06907383058959335
Lower bound to distance -0.06907408346021665
Lower bound to distance -0.06907419362495565


### Learning an MPS representation of the channel
Next we find the best MPS $\ket{c}$ of bond dimension $\chi$ such that $\mathcal{M}(\rho)\approx \mathcal{M}_c(\rho)=\sum_A c_A\rho_A$.
This is based on automatic-differentation library Zigote for minimizing the cost function $||\mathcal{M}(\rho)-\mathcal{M}_c(\rho)||_2^2$

In [163]:
c = Find_c(ψ0,σ,χ);

[ Info: LBFGS: converged after 72 iterations: f = -0.000001441530, ‖∇f‖ = 8.3836e-05


### Inverting the channel
Similary, we can then find the inverse map $\mathcal{M}^{-1}(\rho)=\sum_A d_A \rho_A$, such that $\mathcal{M}^{-1}(\mathcal{M})\approx\mathbf{1}$

In [164]:
d = Find_d(c,2*χ);

random guess261.5936124743024
taylor guess203.22605221297488


[ Info: LBFGS: converged after 494 iterations: f = 0.000000005522, ‖∇f‖ = 8.3744e-05


## Data acquisition
We are ready to perform an experiment on a unknown state and build shadows as MPO

In [165]:
using PastaQ
NM = 10
circuit = randomcircuit(N, depth=1)
#noisemodel1 = (1 => ("depolarizing", (p = 0,)),2 => ("depolarizing", (p = 0.05,)))
ψ = runcircuit(ψ0,circuit);

In [166]:
data = zeros(Int8,(Nu,NM,N)) #Data storage
shadow = Vector{MPO}(undef,Nu*NM)
@showprogress dt=1 for r in 1:Nu
            data[r,:,:] = get_RandomMeas(ψ,u[r],NM) #data acquisition in simulated quantum device
            for m in 1:NM
               shadow[(r-1)*NM+m] = get_ShallowShadow(data[r,m,:],u[r],d,ξ)
            end
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:21


## Extracting observables
Here we focus on XX correlations

In [167]:
O = Vector{Vector{ITensor}}()
for i in 1:N-1
    Ot = Vector{ITensor}()
    push!(Ot,op("X",ξ[i]))
    push!(Ot,op("X",ξ[i+1]))
    push!(O,Ot)
end
O_exact = zeros(N-1)
O_est = zeros(N-1)
for i in 1:N-1
    O_exact[i] += real(trace(apply(O[i],outer(ψ',ψ),apply_dag=false),ξ))
    @showprogress dt=1 for k in 1:Nu*NM
        O_est[i] += real(trace(apply(O[i],shadow[k],apply_dag=false),ξ))/Nu/NM
    end
    println("XX for pair  ",i,i+1)
    println("Exact ", O_exact[i])
    println("Estimated ", O_est[i])
end


Progress: 100%|█████████████████████████████████████████| Time: 0:00:04


XX for pair  12
Exact -0.12363265777828318
Estimated -0.10945861095170155


Progress: 100%|█████████████████████████████████████████| Time: 0:00:04


XX for pair  23
Exact 0.12197957514204251
Estimated 0.13239090428660874


Progress: 100%|█████████████████████████████████████████| Time: 0:00:03


XX for pair  34
Exact 0.41620075379326327
Estimated 0.43330992053386624
